In [8]:
import random
import base64
import re
import math
import heapq
from collections import Counter
import hashlib

In [9]:
class CanalAdaptativo:
    def __init__(Nodo, inicial, aumento, disminucion):
        Nodo.perdida = inicial
        Nodo.aumento = aumento
        Nodo.disminucion = disminucion

    def transmitir(Nodo):

        if random.random() < Nodo.aumento:
            Nodo.perdida += 0.0
        elif random.random() < Nodo.disminucion:
            Nodo.perdida -= 0.0


        if random.random() < Nodo.perdida:

            return True
        else:

            return True
        
class Huffman:
    def __init__(nodo, simbolo, frecuencia):
        nodo.simbolo = simbolo
        nodo.frecuencia = frecuencia
        nodo.izquierda = None
        nodo.derecha = None

    def __lt__(nodo, otro):
        return nodo.frecuencia < otro.frecuencia

def arbolHuffman(texto):
    if not texto:
        return None 

    conteo = Counter(texto)
    print(conteo)
    list_simbolo = [Huffman(simbolo, frecuencia) for simbolo, frecuencia in conteo.items()]
    heapq.heapify(list_simbolo)

    while len(list_simbolo) > 1:
        izquierda = heapq.heappop(list_simbolo)
        derecha = heapq.heappop(list_simbolo)
        fusion = Huffman(None, izquierda.frecuencia + derecha.frecuencia)
        fusion.izquierda = izquierda
        fusion.derecha = derecha 
        heapq.heappush(list_simbolo, fusion)

    return list_simbolo[0]

def GenCodigosHuffman(Nodo, actual, simbol):
    if Nodo.simbolo is not None:
        simbol[Nodo.simbolo] = actual
        return
    GenCodigosHuffman(Nodo.izquierda, actual + '0', simbol)
    GenCodigosHuffman(Nodo.derecha, actual + '1', simbol)

In [10]:
def decodificador(Paquetes, raiz):
    if raiz is None:
        return []
    decoficado = []
    Nodo = raiz
    for empaquetado in Paquetes:
        for bit in empaquetado:
            if bit == '0':
                Nodo = Nodo.izquierda
            else:
                Nodo = Nodo.derecha
            if Nodo.simbolo is not None:
                decoficado.append(Nodo.simbolo)
                Nodo = raiz
                    
    return unir(decoficado)

def unir(texto):
    decodificado = ''.join(chr(int(caracter)) for caracter in texto)
    return decodificado

In [11]:
def Comparador(Paquetes, opcion, raiz, HashOrigen):
    if opcion == 1:
        if raiz is None:
            return ""
        
        huffman = []
        Secundario = [hashlib.sha256(simbolo.encode()).hexdigest() for simbolo in Paquetes]
        derecha = len(HashOrigen)-1
        for idx, subSecundario in enumerate(Secundario):
            HuffmanIDX = BusquedaBinaria(subSecundario, HashOrigen,0,derecha)
            #print(HuffmanIDX)
            if HuffmanIDX is not None:
                huffman.append(Paquetes[idx])
        
        return decodificador(huffman, raiz)
    
    return "Los hashes no coinciden"

def BusquedaBinaria(Secundario, HashOrigen, izquierda, derecha):
    #print(HashOrigen)
    #print(Secundario)
    while izquierda <= derecha:
        #print(derecha)
        medio = (izquierda + derecha) // 2
        #print(medio)
        #print(izquierda)
        if HashOrigen[medio] == Secundario:
            
            return medio
        elif HashOrigen[medio] < Secundario:
            #print(Secundario)
            #print(HashOrigen[medio])
            izquierda = medio + 1
            #derecha = medio - 1
        else:
            derecha = medio - 1
            #izquierda = medio + 1
    #print(hash_originales[medio])
    return None


In [12]:
def codificador2(Paquetes,tamaño,opcion):
    if(opcion == 1):
        raiz = arbolHuffman(Paquetes)
        if raiz is None:
            return "", None
        codigoHuffman = {}
        GenCodigosHuffman(raiz, '', codigoHuffman)
        #EmpaquetadoHuffman = []
        EmpaquetadoHash = []
        codificado = ''.join(codigoHuffman[simbolo] for simbolo in Paquetes)
        for elemento in codificado:
            #print(elemento)
            hash  = hashlib.sha256(elemento.encode()).hexdigest()
            EmpaquetadoHash.append(hash)
        #EmpaquetadoHuffman.append(codificado)
        #EmpaquetadoHash.append(hash)
        return codificado, raiz, codigoHuffman, EmpaquetadoHash
    else:
        return Paquetes
    

def Comparador2(Paquetes, opcion, raiz, HashOrigen):
    if opcion == 1:
        if raiz is None:
            return ""
        
        huffman = []
        Secundario = []
        for elemento in Paquetes:
            for paquetito in elemento:
                hash2 = hashlib.sha256(paquetito.encode()).hexdigest()
                Secundario.append(hash2)
        #print(Secundario)
        derecha = len(HashOrigen)-1
        for idx, subSecundario in enumerate(Secundario):
            HuffmanIDX = BusquedaBinaria(subSecundario, HashOrigen,0,derecha)
            #print(HuffmanIDX)
            if HuffmanIDX is not None:
                huffman.append(Paquetes[idx])
        
        return decodificador(huffman, raiz)
    
    return "Los hashes no coinciden"

In [13]:
def fuente(filePpath):
    try:
        with open(filePpath, "r", encoding="utf-8") as file:
            text = file.read()
            binarioTexto = ''.join(format(ord(caracter), '08b') for caracter in text)
            Paquete = [binarioTexto[i:i+8] for i in range(0, len(binarioTexto), 8)]
        return Paquete
    except FileNotFoundError:
        print(f"El archivo '{filePpath}' no se encontró.")
        return []
    
def codificador(Paquetes,tamaño,opcion):
    if(opcion == 1):
        raiz = arbolHuffman(Paquetes)
        if raiz is None:
            return "", None
        codigoHuffman = {}
        GenCodigosHuffman(raiz, '', codigoHuffman)
        Empaquetado = [Paquetes[i:i+tamaño] for i in range(0, len(Paquetes), tamaño)]
        EmpaquetadoHuffman = []
        EmpaquetadoHash = []
        for Paquetes in Empaquetado:
            codificado = ''.join(codigoHuffman[simbolo] for simbolo in Paquetes)
            hash  = hashlib.sha256(codificado.encode()).hexdigest()
            EmpaquetadoHuffman.append(codificado)
            EmpaquetadoHash.append(hash)
        return EmpaquetadoHuffman, raiz, codigoHuffman, EmpaquetadoHash
    else:
        return Paquetes
    
def sistemaAdaptativo(canales, paquetes):
    intentos = 0
    actual = random.choice(canales)
    recibidos = []
    perdidos = []
    for paquete in paquetes:
        exito = actual.transmitir()
        if exito:
            recibidos.append(paquete)
        else:
            perdidos.append(paquete)
            print("Paquetes perdidos:")
            print(perdidos)
            while perdidos:
                perdido = perdidos.pop(0)
                exito = actual.transmitir()
                intentos += 1
                if exito:
                    recibidos.append(perdido)
        actual = random.choice(canales)
    print("Paquetes recibidos:")
    print(recibidos)
    print(f"Se transmitieron los {len(paquetes)} paquetes con {intentos} intentos.")
    if intentos > 0:
        print("Se requirió retransmisión debido a pérdida de paquetes.")

# Función para reconstruir el libro con los cambios solicitados
def destino(decoded_text, output_file_path):
    decoded_text_str = ''.join(str(item) for item in decoded_text)
    with open(output_file_path, 'w', encoding='utf-8') as f:
        f.write(decoded_text_str)
    print(f"Texto reconstruido guardado en '{output_file_path}'.")

def handshake(Paquetes,tamaño, opcion):
    if opcion == 1:
        canales = [CanalAdaptativo(inicial=0.0, aumento=0.0, disminucion=0.0) for _ in range(1)]
        codificado, root, huffmanCodes, hash = codificador2(Paquetes,tamaño, opcion)
        sistemaAdaptativo(canales, hash)
        #print(hash)
        decoficado = Comparador2(codificado , opcion, root,hash)
        print(f"Los códigos Huffman son: {huffmanCodes}")
        print(f"Texto original: {Paquetes}")
        print(f"Texto codificado: {codificado}")
        print(f"Texto hash: {hash}")
        print(f"Texto decodificado: {decoficado}")
        return decoficado
    
    else:
        canales = [CanalAdaptativo(inicial=0, aumento=0, disminucion=0) for _ in range(1)]
        codificado = codificador(Paquetes,tamaño, opcion)
        sistemaAdaptativo(canales, codificado)
        decoficado = Comparador(codificado , opcion, None)
        return decoficado

In [14]:
inputFilePath = "test_text.txt"
outputFilePath = "test_text_reconstructed.txt"
Paquetes = fuente(inputFilePath)
ascii = [int(binary, 2) for binary in Paquetes]
book = handshake(ascii,200,1)
destino(book, outputFilePath)

Counter({32: 66, 101: 40, 116: 37, 97: 32, 110: 31, 111: 30, 105: 30, 108: 25, 115: 21, 114: 19, 104: 16, 99: 15, 117: 12, 100: 12, 109: 10, 119: 8, 102: 8, 121: 7, 103: 6, 10: 6, 112: 5, 98: 5, 107: 4, 44: 3, 46: 3, 118: 3, 72: 2, 66: 1, 113: 1, 59: 1})
Paquetes recibidos:
['...']
Se transmitieron los 1983 paquetes con 0 intentos.
Los códigos Huffman son: {101: '000', 115: '0010', 112: '001100', 66: '00110100', 72: '00110101', 46: '0011011', 117: '00111', 100: '01000', 10: '010010', 103: '010011', 108: '0101', 44: '0110000', 107: '0110001', 121: '011001', 99: '01101', 105: '0111', 111: '1000', 110: '1001', 102: '101000', 119: '101001', 104: '10101', 97: '1011', 32: '110', 116: '1110', 114: '11110', 98: '1111100', 59: '111110100', 113: '111110101', 118: '11111011', 109: '111111'}
Texto original: [66, 101, 114, 103, 32, 97, 108, 119, 97, 121, 115, 32, 115, 112, 111, 107, 101, 32, 113, 117, 105, 101, 116, 108, 121, 44, 32, 112, 111, 108, 105, 116, 101, 108, 121, 44, 32, 97, 110, 100, 32,